    This file is part of oxtr-ko_2017.03.
    Copyright (C) 2018  Emir Turkes

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

    Emir Turkes can be contacted at eturkes@bu.edu

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# load_data.py
import pymice as pm

# Files relevent to Habituation 1 (hab1) period.
dataFiles = ['../data/comp+old-behav-flex/2017-03-20 11.02.07.zip', \
             '../data/comp+old-behav-flex/2017-03-20 19.58.27.zip', \
             '../data/comp+old-behav-flex/2017-03-21 09.53.27.zip']

# Merge the data.
loaders = [pm.Loader(filename) for filename in dataFiles]
data = pm.Merger(*loaders)

print("Done loading data.")

for mouse in sorted(data.getGroup()):
    print(mouse)
print("Ignore Pump group, it is related to another experiment.")

# Read in hab1 period from timeline.ini.
timeline = pm.Timeline('../timeline/comp+old-behav-flex_timeline.ini')
start, end = timeline.getTimeBounds(timeline.sections()[0])
print("%s:\t%s - %s" % (timeline.sections()[0], start, end))

# Check for any problems (indicated in the log) during the period of
# interest.
start, end = timeline.getTimeBounds(PHASES)

dataValidator = pm.DataValidator(pm.PresenceLogAnalyzer())
validatorReport = dataValidator(data)

noPresenceProblems = pm.FailureInspector('Presence')

if noPresenceProblems(validatorReport, (start, end)):
    print("Presences OK.")
    
# Find total visits and duration by group.   

visitsByGroup = dict()
durationByGroup = dict()

for mouse in sorted(data.getGroup()):
    if mouse != 'Cage9 Pump':
        mice = [data.getAnimal(m) for m in data.getGroup(mouse).Animals]
        visits = data.getVisits(mice=mice, order='Start')
        visitorNames = [v.Animal.Name for v in visits]

        visitsByGroupCage = 0
        durationByGroupCage = 0

        if mouse[6:] not in visitsByGroup:
            visitsByGroup[mouse[6:]] = visitsByGroupCage
        if mouse[6:] not in durationByGroup:
            durationByGroup[mouse[6:]] = durationByGroupCage

        for mouse in set(visitorNames):
            visitsByGroupCage = visitsByGroupCage + visitorNames.count(mouse)
        for mouse in set(visitorNames):
            for timePeriod in timeline.sections():
                if timePeriod != 'Habituation 1':
                    start, end = timeline.getTimeBounds(timePeriod)
                    timeVisits = data.getVisits(mice = mice, \
                                                start = start, \
                                                    end = end)
                    for i in range(1, len(timeVisits)):
                        durationByGroupCage = durationByGroupCage + \
                        timeVisits[i].Duration.total_seconds()
                
        if mouse[-2:] in visitsByGroup:
            visitsByGroup[mouse[-2:]] = visitsByGroup[mouse[-2:]] \
            + visitsByGroupCage
        if mouse[-2:] in durationByGroup:
            durationByGroup[mouse[-2:]] = durationByGroup[mouse[-2:]] \
            + durationByGroupCage

print('%s: %d visits' % ('WT', visitsByGroup['WT']))
print('%s: %d visits' % ('HT', visitsByGroup['HT']))
print('%s: %d visits' % ('KO', visitsByGroup['KO']))

print('%s: %d seconds' % ('WT', durationByGroup['WT']))
print('%s: %d seconds' % ('HT', durationByGroup['HT']))
print('%s: %d seconds' % ('KO', durationByGroup['KO']))